In [ ]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

### Script to write csv_file with every nifti for DeepOncology repertory

#### - Read JSON dataset 

In [ ]:
json_path = '/media/deeplearning/LACIE SHARE/good/ahl_nifti_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

#### - Get the list of every nifti in the directory 

In [ ]:
nifti_directory = '/media/deeplearning/LACIE SHARE/good//NIFTI'
liste = os.listdir(nifti_directory)
liste_nifti = []
for nifti in liste : 
    if 'nii' in nifti : 
        liste_nifti.append(os.path.join(nifti_directory, nifti))

In [ ]:
len(liste_nifti)

#### - Add nifti PT CT MASK for every data/study in dataset 

In [ ]:
for data in dataset: 
    uid = data[4]
    #print(uid)
    subliste = []
    for nifti in liste_nifti : 
        if uid in nifti : 
            subliste.append(nifti)
    subliste.sort()
    #print(subliste)

    if len(subliste) == 3 :   #   3 if mask 
        for i in range(len(subliste)):
            data.append(subliste[i])
    

In [ ]:
dataset[0]

In [ ]:
error = []
for serie in dataset : 
    if len(serie) != 11: 
        print(serie)
        error.append(serie)


for err in error : 
    dataset.remove(err)

In [ ]:
print("nombre de study : ", len(dataset))

#### - Create folder with patient_id and put nifti into it 

In [ ]:
directory =  '/media/deeplearning/LACIE SHARE/good/NIFTI'
for serie in dataset: 
        #patient_name = serie[0]
    patient_id = serie[6]
    pet_type = serie[5]
    study_uid = serie[4]
    nifti_ct = serie[-3]
    nifti_pet = serie[-2]
    nifti_mask = serie[-1] #if pet0


    folder = directory+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "Déjà creer => already folder for ", pet_type)

    ct_path = nifti_ct
    pt_path = nifti_pet
    mask_path = nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct.split('/')[-1]
    new_pet_path = subfolder+'/'+nifti_pet.split('/')[-1]
    new_mask_path = subfolder+'/'+nifti_mask.split('/')[-1]
    print(new_ct_path)
    print(new_pet_path)
    print(new_mask_path)
    
    try : 

        os.rename(ct_path, new_ct_path)
        os.rename(pt_path, new_pet_path)
        os.rename(mask_path, new_mask_path)
            

        serie[-3] = new_ct_path
        serie[-2] = new_pet_path 
        serie[-1] = new_mask_path

    except Exception as err : 
        print(err)
    

#### - Write csv_file 

In [ ]:
directory = '/media/deeplearning/LACIE SHARE/good/NIFTI'
filename = 'AHL2011_NIFTI_last.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT NAME", "PATIENT_ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK"])
    for serie in dataset: 
        csv_writer.writerow([serie[6], serie[6], serie[4], serie[5], serie[-3], serie[-2], serie[-1]])
